# Image Alignment and Stitching

<center><img src="figs/stitching_alcatraz.jpeg" width=1100px alt="default"/></center>

Thank you for completing the Anonymous Feedback Form: https://forms.gle/9694KDv1dzQm6dJ28 (URL in slack)

# Last Unit

- 01: Image Formation Models (Ch. 2)
- 02: Image Operators and Transforms (Ch. 3)
- **03: Feature Detection, Matching, Segmentation (Ch. 7)**
- 04: Image Alignment and Stitching (Ch. 8)
- 05: 3D Reconstruction (Ch. 13)

We saw how to find features:
- which can be "matched" for a downstream task,
- which are interesting in themselves (boundaries of objects).

# Last Unit

- 01: Image Formation Models (Ch. 2)
- 02: Image Operators and Transforms (Ch. 3)
- **03: Feature Detection, Matching, Segmentation (Ch. 7)**
- 04: Image Alignment and Stitching (Ch. 8)
- 05: 3D Reconstruction (Ch. 13)

We saw how to find features:
- which can be "matched" for a **downstream task**,
- which are interesting in themselves (boundaries of objects).

# This Unit

- 01: Image Formation Models (Ch. 2)
- 02: Image Operators and Transforms (Ch. 3)
- 03: Feature Detection, Matching, Segmentation (Ch. 7)
- **04: Image Alignment and Stitching (Ch. 8)**
- 05: 3D Reconstruction (Ch. 13)

# Definitions

$\color{#EF5645}{\text{Matching features}}$ means to find corresponding feature locations in different images.

$\color{#EF5645}{\text{Image alignment}}$ means to find the geometric transformation that can transform an image into another. This can be done using matched features.

$\color{#EF5645}{\text{Image stitching}}$ means "gluing" images together in a consistent way, for example to form a panorama. This can be done using image alignment.

<center><img src="figs/03_main2.png" width=600px alt="default"/></center>

# Image Alignment and Stitching

Your Presentation!